In [ ]:
from datasets import load_dataset
from random import randint
import json

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("lmsys/lmsys-chat-1m")['train']
sample_idxs = [randint(0, len(ds)) for _ in range(300)]
samples = [ds[i] for i in sample_idxs]
single_turn_samples = [s for s in samples if len(s['conversation']) == 2]
prompts = [s['conversation'][0]['content'] for s in single_turn_samples]
with open('prompts.json', 'w') as f:
    json.dump(prompts, f, indent=2)

In [ ]:
import json
prompts = json.load(open("prompts.json"))[:100]
server_url = "https://e1f18acc28cf24eea6.gradio.live/"

In [ ]:
from openai import OpenAI
def get_response_standard(prompt):
    system_prompt = "You are a helpful assistant."
    client = OpenAI()
    completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ],
    temperature=1.0,
    top_p=0.7,
    max_tokens=512,
    )

    return completion.choices[0].message.content



from gradio_client import Client
def get_response_agent(prompt):
    client = Client("https://e1f18acc28cf24eea6.gradio.live/")
    result = client.predict(
            model_selector="react-agent",
            text=prompt,
            api_name="/add_text_1")
    out = client.predict(
    temperature=1.0,
    top_p=0.7,
    max_new_tokens=512,
    api_name="/bot_response_2"
    )
    return out[0][1]


In [ ]:
import tqdm
responses = []
for prompt in tqdm.tqdm(prompts):
    agent_response = get_response_agent(prompt)
    standard_response = get_response_standard(prompt)
    responses.append({"prompt": prompt, "agent_response": agent_response, "standard_response": standard_response})

In [ ]:
with open("responses.json", "w") as f:
    json.dump(responses, f, indent=2)